In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

import rnn
import train
import brainfuck

In [2]:
embedding_size = 10
hidden_size = 35
output_size = len(rnn.char.keys())
n_layers = 2
batch_size = 64
GAMMA = 0.99 # for exponential moving avarage constant from paper




In [3]:
print(rnn.token_num)

9


In [4]:
model = rnn.BFgen(rnn.token_num, embedding_size, hidden_size, output_size, n_layers, batch_size)

In [5]:
print(train.objective_PG(model, lambda x : np.array([1] * model.batch_size)), model.entropy)

(Variable containing:
-133.2633
[torch.FloatTensor of size 1]
, Variable containing:
 13290.1924
[torch.FloatTensor of size 1]
)


In [6]:
print(train.objective_PQT(model))

Variable containing:
-208.3982
[torch.FloatTensor of size 1]



In [7]:
print(model.pqt_programs)

['+-<-.<[[.<+[+-].]<].<]->>--[]<>..+<>-]]]-..,.->,<[+>><>[,-[+][-<>+-,.-.-+<]>],++,[,-<[[+<++[><><].<<'
 '>-,[-,+,..-,,].++>>,.><.]-..,<]>-]-],[<>>.->[[-,<.<[<,<[<<+-<,.>,.]+,.[[-+.<.>-[.-<--][]><>,.,,].]<-'
 '->-+]]]<.]-<+,]-]]-<..<,...]>-],.].-<,]]],..>-[,<]+>>[--.><-[,>+>--,]][,<.,-[.[-]+]+.->++<+>+<<<[,>>'
 ',<]<+>.--<,+]-,[+.-][[.<+,+[,....>.>,[[-[>>,[.<>..>><<<.,>>].,]<[.>,+-.,>,[>+,>-,>>.]]<-,,>-+<,[>>,['
 '>[+,,>[[>]+>].]][[>[,>[<.,,.],>[<>[,-<[[]+,++.<+[.[>-]<][]+>[,><[,-.[>]>,]+.[[<<]-][++<+[<>]<<-[[+>-'
 ',.]]<<[.-+].-.].[>+.-+[.>.+[+,-+++],[.,>>--<+-,]],]]+[.<.[[,<[-.-[<>-<[++<<<+,[>+++><,[-..->.],+-]+['
 '<[,[>>>-.]<,+]<+>,<]<,,+>+>>->>><-.<>.]+-][,+[,,+>[.+]<],.+][>+-+[[><[.>+[,.>><[.<>.[[[<[<[.+-.<],<]'
 '-]-+[[,]--+.,+<<[>..>-,-,+>--<].,[.>+<+>].<<<[>,]+.]].]]+]<+,<]-<+,+.[<-[[-+,.,.<+><][],+>[<]>>.<,<>'
 '.[<>.[,+<<>[[<--+,[[,,,]],,><[>+[---.]-<[.[.[][]>[.]+,.+<-][+,>,-<>].<[>+--<>.,-<>+-[>-.>>+]]+,]]<+['
 '>,[[.][,][----],.---[>->[,><<+-.[+,<-[--.[[-+><.-.-<+-.>]-->-<

In [8]:
bfAdd = ",>++++++[<-------->-],[<+>-]<++++++++."
bfAdd2 = ",>++++++[<-------->-],[<+>-]<."
bfAdd3 = ",>++++++[<-------->-],[<+>-]<++..."
bfAdd4 = ",>++++++[<-------->-],[<+>-]<+++............"
addBatch = [bfAdd, bfAdd2, bfAdd3, bfAdd4]
bfAddIn = ['11', '12', '23']
bfAddOut = ['2','3','5']

In [9]:
reward = train.batch_reward(bfAddIn, bfAddOut)

In [10]:
reward(addBatch)

array([  76.5,   76.8,  -77.1, -768.3])

In [11]:
reverse_in = ["a", "ab", "abc", "abcd", "abcde"]
reverse_out = ["a", "ba", "cba", "dcba", "edcba"]
reverse_reward = train.batch_reward(reverse_in, reverse_out, scaling_factor=1./3840.)

In [12]:
model.clear_pq()
#train.train_pqt_pg(model, reverse_reward, NPE=1000000)

In [13]:
import bfCompiler
brainfuck.BF("-+[,-><<<.+.,+<-,><+-..]-<..+.>>[[<[-<+-.]-]+>+.+>-.]-,+>]][>]+]..><->][,[>,..<,<[[]][[++<>-],+..+--", "abc")

'\xff\xff'

In [14]:
expected_code = "><.><"
program_batch_code = [".....", "..<..", "[][][][][[][[][][][][][][]]]", "]"]

In [15]:
simple_rew = train.simplified_batch_reward(expected_code)

In [16]:
simple_rew(program_batch_code)

array([ 0.996875  ,  0.99609375, -1.        ,  0.19921875])

In [17]:
training_expected_code = ">>>.." * 20
print len(training_expected_code),training_expected_code

100 >>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..>>>..


In [18]:
training_rew = train.simplified_batch_reward(training_expected_code)
training_rew(program_batch_code)

array([0.04988281, 0.04988281, 0.27890625, 0.00996094])

In [19]:
def ultra_simple_rew(expected_code):
    def hamming_distance(seq1, seq2):
        assert len(seq1) == len(seq2)
        return sum(elem1 != elem2 for elem1, elem2 in zip(seq1, seq2))
    
    code_len = float(len(expected_code))
    def reward_code(generated_code_batch):
        rewards = np.array(map(lambda x: 1 - 2*(hamming_distance(x, expected_code) / code_len), generated_code_batch))
        return rewards
    return reward_code

from itertools import takewhile
def prefix_rew(expected_code):
    def common_prefix_len(seq1, seq2):
        return len(list(takewhile(lambda (a, b): a == b, zip(seq1, seq2))))
    
    code_len = float(len(expected_code))
    def reward_code(generated_code_batch):
        rewards = np.array(map(lambda x: 2*(common_prefix_len(x, expected_code) / code_len) - 1, generated_code_batch))
        return rewards
    return reward_code

In [20]:
simple_rew = prefix_rew(training_expected_code)
sample1 = ">" * 100
sample2 = "." * 100
sample3 = "[]" * 50
sample4 = "[]><+-.,[]"  * 10
samples = [sample1, sample2, sample3, sample4]

In [21]:
simple_rew(samples)

array([-0.94, -1.  , -1.  , -1.  ])

In [25]:
model.clear_pq()
model.init_weights()
train.train_pqt_pg(model, simple_rew, exp_code=training_expected_code, NPE=10000000)

Epoch Obj  Sample
----- ----- ------
Variable containing:
-0.2930
[torch.FloatTensor of size 1]

Variable containing:
-209.6326
[torch.FloatTensor of size 1]

Variable containing:
 205.6150
[torch.FloatTensor of size 1]

-0.9956467793999384
pqt [('><-<[-.].[[-.[>]]-<[[]<[,->>,.,]<[[]]<.+<>,-,<<.[<<-+].>]-,-,[]+.,]>,-<>>+<<-<,.-+>+>],[<,[,[,<<-+,[', -0.98), ('><-<<.,>-[.[,].+<[<>++>>+[]+[[.[>,++,>.][[.[,>.>]<-+]<<,.<+-<,-]--+<,]],.-][[,],,..<.,+>+],,[>>[[-<.', -0.98), ('>.]<...-[]-[+.]]]]>.[-[].[[][.+]]]<+-[.>[[<..,,.[].+.],+-.-]-[,]]-[<<+]]--].>]<]]+-,[[.-,][>.+.<.-+-', -0.98), ('><+,<[-]<,,.>+[>[,[]<---,+,.]]>.<]<[>+,,-],.,]]<,]]<-.><-,+][>.-].[[.->+,[,+,>.+>[,.+.+,[,>.-[>-<-->', -0.98), ('><+<]>][,><]..+,+>[><-><.[>,,<[+,+[[[.>-.<,[-,+>.,+>,[],.[,-,]-<[+.-.]]<.<[><,<[,[][+->.,[->,+><][<[', -0.98), ('>[-<<-[>+,[-[.<>>..[-[]->[>,.<[<+.>,<]<[,,<[[]<,-+>-,<.>-,<+>[+,+<->.<<]<[,,]--[.-,[>+<<,++>..,][],<', -0.98), ('>],><+<>[,>]]<.[,.>+><,>-.+<->][<-[+]+[>>>+.[],,,+].,-+-]-]-,,]+.[]+<>,>+,

Variable containing:
-0.2671
[torch.FloatTensor of size 1]

Variable containing:
-195.6641
[torch.FloatTensor of size 1]

Variable containing:
 199.0743
[torch.FloatTensor of size 1]

-0.9965811534842194
pqt [('>>>..<-+>++<]>-[<]-]+><]+.<..[<].+-[[-<].,]<-<+>><-+,.<[-><.->--,[.+,[+.,+[]<.-[.+->+++..[[+-<,.>-<]', -0.9), ('>>>..]+..,->+-<,]<,,]<,[]+<<>+.+<+>.,>-+,+>+.[-[+.].,]-[--[+>-,+.[--<+.>[<>]-[>-.,.].,<->,...->[]+>+', -0.9), ('>>>..[[[+]>-+,>>+.,+<].-.,-[[>+]>-<-+[[]].>>-<]-+..-[]-.,]<>+>]-[->++,]><,+],.,,,>]<,<]<<-,>>[,-<[+]', -0.9), ('>>>..+[]+,.<[>.[..-<.>,,+,<>[[[+<,-[>>[.<]+-<.>,>---+,-,,,>]]+<+[+<+>]+],[<[]<-<],]<],]>.<<+.][-[,[]', -0.9), ('>>>..<->.+>,[<-],-+.]+[[>+<]+>+.,-[<[]+].>]<][[..,>]>[-]+[>]<,,<]-.+<[<]-,<>>[<]>>-[><.[>[<,]>,]++.+', -0.9), ('>>>..]]][-++>,[[]+-.]-,,<..],.[+<-][[,--++],+-<.--[,[+-><<,.,<..,...]]]--+>]+-]<,]-++,>><><-<].,.]>.', -0.9), ('>>>..],[>,,[,>>.,]-[>,[>+<.<>++>>+,[.[.<<<[<<<],,[-+]>,+>-,-<[..++<],+-++,>>+]>.>-[+-[.,.><]-[+[.,,,', -0.9), ('>>>..

Variable containing:
1.00000e-03 *
 -2.1889
[torch.FloatTensor of size 1]

Variable containing:
-193.1145
[torch.FloatTensor of size 1]

Variable containing:
 198.0577
[torch.FloatTensor of size 1]

-0.9965626556189363
pqt [('>>>..[+-><><-]-,,+.-+,+,[.,---,]<+><,.<[,,[-,..].][>+,]><]+]+,[[[[,-],[[[+-,<<+,+][[]]>>]<.>.],,<>[[', -0.9), ('>>>...-.>[<,]]+++.--[,---.++[].<<]-+.<],<<,].[[<<+.<>]>.][-,>-+,><,-+><><,.>,+[]>>+[-+[<[+,.+++-,>[]', -0.9), ('>>>...>[><-.]+.[]<[-<[-<->]]<]+,<.[+<<,+]<>]]<[,[+,,]<+>,-><->[.]+<]<,-.,,,]+++<]+[.,>[.[<+,<.+..,[]', -0.9), ('>>>..-<[]>,++<<-.+.[.[,[+]<<,,++<.].>]-]><.,<,].<[,>].>[<..,>-+-][[,<->++>-.-,,-[<[,--<+<[-++>-<---[', -0.9), ('>>>...-[>[+[[+->[[<,>[,[>,+[[+],-.>+,]]<-]+][],++<-]>]<++<<[[-][[+-<,...,<]>[-,<.>,>[+[>+>[<,].--[>-', -0.9), ('>>>..[-.<+.-+-+,+-[.--+[].+>,->-]]+.<-.,<,++>]<<,<,<>-+]<.-<[[...,>+,>>>++[,.+]]]>[-[[<<.[[+>>,>.-+]', -0.9), ('>>>..>,++.>[>-<],+--<-,,[+...-[].]<]>-[,[>.<.[.-.<-]->+.[,[]<].<.<<.].>>>[.>],,,-]--+>[>.[[<[+>[<+-[',

Variable containing:
1.00000e-02 *
 -8.5796
[torch.FloatTensor of size 1]

Variable containing:
-187.8958
[torch.FloatTensor of size 1]

Variable containing:
 187.5480
[torch.FloatTensor of size 1]

-0.9959435971935923
pqt [('>>>..<+<<]>]>.,+-[,]..[[..<,>.]>[<].>-+-[,>.-<<[+[]+[.]<->>,+-[],.->><-[.><][<.,].,+-<++[[.-,[-+>+--', -0.9), ('>>>..+>+]-,<>+[-.+]-,>].[<<<>+.<],,.]-]].,<]]..>.,.-,[],+<.,<+>>.-]>,[-,]],+[-.,-...+]-<.[.>[,<--++,', -0.9), ('>>>..+,->]].]>]<+,[]>-+]<+<]<]]<]<.,]-,<+]-[,-<+]-,-.+<.>,+,,>++]-<.+[<],<.[>]]>+,->,.>+.-[-+>,<>.<>', -0.9), ('>>>...---.,]<>[<+-]<+,.+[<>][].]>>,..<,>+][.>>.[++<+-+[]>,><--,[,++<].].,>.]>.->-,],+,<.[,+-[<[<-+..', -0.9), ('>>>..>+->+[-<--+],.<><.+><[>][+.>,[++][-]<[]+[-]<>,.>[><.>[<+][].-[+[,.+<<>++-.-+,<]>.+,,+<]><-[.[-<', -0.88), ('>>>..>[-.,,+.<.[,-+.,>+.[.-+][>+]<,[<.-++.+,<]-][><+,[<+-]]>],-,<-+.-[+><]<-,+[->.[[[.>[[]-].,]>]],>', -0.88), ('>>>..><,+..>]<]>-].-]<]<[[+.-.[<-+>]>.<.<[.[++<.>>,.-[]>.++]+++++-->,>]<.+-,[+,[-+<]>>,[+>-]-<.+[.>-

Variable containing:
1.00000e-02 *
  4.2503
[torch.FloatTensor of size 1]

Variable containing:
-185.0038
[torch.FloatTensor of size 1]

Variable containing:
 182.7954
[torch.FloatTensor of size 1]

-0.9971844080875636
pqt [('>>>..-[<,-+<+>.<,+<>[,[<<,<[,<]-+-.<]+.>,]-[-.,-]][][-<+]><][]],,<,,+>..>],]<,[+>-]><,--[,][-[<-.].-', -0.9), ('>>>..<.+-][-.>.<+].+,<<><[+,>+.<.<],]<]<+.,+>..,++.[[<+>>>>->>],,>.++.>+-,++<,+.,,+]..+]+.++,+.<>-+<', -0.9), ('>>>..+]+.+-,[,-.-.[[[[].-<><++[,<>],,.>].+.--++,-<<+<[...<[+-<-->>.--,<.<<<[+-,++-,-+.<>.<+<-,+.<<[[', -0.9), ('>>>..>][,]][<.[]<,+]<<]>].<+[,<.<+,.<.-,<,-]-[>->.><,<,-,]+<[.,,>-]<,+]>+[<,+<>[+.]--,<[,],.-],<--.>', -0.88), ('>>>..>,++.>[>-<],+--<-,,[+...-[].]<]>-[,[>.<.[.-.<-]->+.[,[]<].<.<<.].>>>[.>],,,-]--+>[>.[[<[+>[<+-[', -0.88), ('>>>..>[-.,,+.<.[,-+.,>+.[.-+][>+]<,[<.-++.+,<]-][><+,[<+-]]>],-,<-+.-[+><]<-,+[->.[[[.>[[]-].,]>]],>', -0.88), ('>>>..>+->+[-<--+],.<><.+><[>][+.>,[++][-]<[]+[-]<>,.>[><.>[<+][].-[+[,.+<<>++-.-+,<]>.+,,+<]><-[.[-

KeyboardInterrupt: 

In [23]:
print(model.get_probs(training_expected_code))

Variable containing:
 0.1178
[torch.FloatTensor of size 1]



In [24]:
print(model.sample())

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
